# Métrica 3_4_04 - “Inmuebles adquiridos por compraventa en copropiedad o por un tercero”

| Escenario                                                                                                                                                | Condición | Resultado        |
| -------------------------------------------------------------------------------------------------------------------------------------------------------- | --------- | ---------------- |
| No hay bienes inmuebles                                                                                                                                  | —         | ⚪ **SIN_DATO**   |
| Todos los inmuebles por compraventa están a nombre del declarante                                                                                        | —         | 🟢 **CUMPLE**    |
| Hay inmuebles adquiridos por compraventa a nombre de tercero / pareja / copropiedad y su número está dentro del rango esperado según el nivel jerárquico | —         | 🟢 **CUMPLE**    |
| Número fuera del rango esperado (por arriba del Q3 o umbral)                                                                                             | —         | 🔴 **NO_CUMPLE** |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "3_4_04_INMUEBLES_COMPRAVENTA_COPROPIEDAD_TERCEROS"
BATCH_SIZE = 10000

UMBRAL_POR_NIVEL = {
    "OPERATIVO": (0, 1),
    "ENLACE": (0, 2),
    "JEFE_DEPARTAMENTO": (0, 3),
    "SUBDIRECTOR": (0, 4),
    "DIRECTOR": (0, 4),
    "TITULAR": (0, 5)
}

TITULARES_TERCERO = {
    "TERCERO", "TERCEROS", "PAREJA", 
    "DECLARANTE Y PAREJA", "PAREJA Y DECLARANTE",
    "DECLARANTE Y TERCERO", "COPROPIEDAD",
    "TITULARIDAD COMPARTIDA"
}

def extraer_lista(doc, path):
    try:
        partes = path.split(".")
        actuales = [doc]
        for p in partes:
            siguientes = []
            for a in actuales:
                if isinstance(a, dict):
                    v = a.get(p)
                    if v is None: continue
                    if isinstance(v, list): siguientes.extend(v)
                    else: siguientes.append(v)
                elif isinstance(a, list):
                    for sub in a:
                        if isinstance(sub, dict):
                            v = sub.get(p)
                            if v is None: continue
                            if isinstance(v, list): siguientes.extend(v)
                            else: siguientes.append(v)
            actuales = siguientes
        return [x for x in actuales if isinstance(x, dict)]
    except:
        return []

def extraer_valor(doc, path):
    try:
        partes = path.split(".")
        a = doc
        for p in partes:
            if isinstance(a, dict):
                a = a.get(p)
            elif isinstance(a, list) and len(a) > 0:
                a = a[0].get(p)
            else:
                return None
        return a
    except:
        return None


def evaluar_metrica(doc):
    bienes = extraer_lista(doc, "declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble")
    nivel = str(extraer_valor(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivel") or "").upper()

    if not bienes:
        return "SIN_DATO"
    if not nivel:
        return "SIN_DATO"

    # contar inmuebles por compraventa no a nombre del declarante
    n_tercero = 0
    for b in bienes:
        titular = str(b.get("titular", "")).upper()
        forma = str(b.get("formaAdquisicion", "")).upper()

        if any(k in forma for k in ["COMPRA", "COMPRA-VENTA", "COMPRAVENTA"]):
            if titular in TITULARES_TERCERO:
                n_tercero += 1

    if nivel not in UMBRAL_POR_NIVEL:
        return "SIN_DATO"

    q1, q3 = UMBRAL_POR_NIVEL[nivel]

    if n_tercero < q1 or n_tercero > q3:
        return "NO_CUMPLE"

    return "CUMPLE"


def procesar_metrica():
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    db = client[DB_NAME]
    src = db[SOURCE_COLLECTION_NAME]
    tgt = db[METRICS_COLLECTION_NAME]

    total_docs = src.estimated_document_count()
    print(f"🚀 Procesando {METRIC_ID} con {total_docs:,} documentos...\n")

    skip = 0
    procesados = 0
    lote = 0

    try:
        while True:
            cursor = list(src.find(
                {},
                {
                    "_id": 1,
                    "declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble": 1,
                    "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivel": 1
                }
            ).skip(skip).limit(BATCH_SIZE))

            if not cursor:
                break

            operaciones = []
            for doc in cursor:
                try:
                    resultado = evaluar_metrica(doc)
                except Exception as e:
                    resultado = "SIN_DATO"
                    print(f"[Error doc {doc.get('_id')}] {e}")

                operaciones.append(UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                ))

            if operaciones:
                tgt.bulk_write(operaciones)
                procesados += len(operaciones)
                lote += 1
                print(f" 🔄 Lote {lote:,} → {procesados:,}/{total_docs:,} procesados", flush=True)

            skip += BATCH_SIZE

        print("\n✅ Procesamiento completado.")
        print(f"Total final: {procesados:,}/{total_docs:,} documentos actualizados.")

    except Exception:
        traceback.print_exc()
    finally:
        client.close()
        print("🔒 Conexión cerrada.\n")


if __name__ == "__main__":
    procesar_metrica()


🚀 Procesando 3_4_04_INMUEBLES_COMPRAVENTA_COPROPIEDAD_TERCEROS con 8,942,592 documentos...

 🔄 Lote 1 → 10,000/8,942,592 procesados
 🔄 Lote 2 → 20,000/8,942,592 procesados
 🔄 Lote 3 → 30,000/8,942,592 procesados
 🔄 Lote 4 → 40,000/8,942,592 procesados
 🔄 Lote 5 → 50,000/8,942,592 procesados
 🔄 Lote 6 → 60,000/8,942,592 procesados
 🔄 Lote 7 → 70,000/8,942,592 procesados
 🔄 Lote 8 → 80,000/8,942,592 procesados
 🔄 Lote 9 → 90,000/8,942,592 procesados
 🔄 Lote 10 → 100,000/8,942,592 procesados
 🔄 Lote 11 → 110,000/8,942,592 procesados
 🔄 Lote 12 → 120,000/8,942,592 procesados
